In [ ]:
import fiona
import math

import matplotlib.pyplot as plt

import cartopy.crs as ccrs
from cartopy.io import shapereader
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import cartopy.io.img_tiles as cimgt

from haversine import distance

In [ ]:
from shapely.geometry import shape

class Route:
    def __init__(self, filepath):
        self.filepath = filepath
        self.coordinates = None
        self.bounds = None

    @property
    def coords(self):
        if not self.coordinates:
            self.load_file()
        return self.coordinates

    def shape(self):
        data = {'type': 'MultiLineString', 'coordinates': self.coords}
        return shape(data)

    def load_file(self):
        layer = fiona.open(self.filepath, layer='tracks')
        self.bounds = layer.bounds
        geom = layer[0]
        self.coordinates = geom['geometry']['coordinates']

In [ ]:
class Routes:
    def __init__(self):
        self.routes = []
        self.bounds = [100.0, 100.0, -100.0, -100.0]

    def add(self, route):
        route.load_file()
        self.routes.append(route)
        self.bounds = [
            min(self.bounds[0], route.bounds[0]),
            min(self.bounds[1], route.bounds[1]),
            max(self.bounds[2], route.bounds[2]),
            max(self.bounds[3], route.bounds[3]),
        ]


In [ ]:
routes = Routes()

from glob import glob

for path in glob('routes/*.gpx'):
    routes.add(Route(path))

print(f'{len(routes.routes)} routes loaded')

In [ ]:
def make_map(projection=ccrs.PlateCarree()):
    fig, ax = plt.subplots(figsize=(9, 13),
                           subplot_kw=dict(projection=projection))
    gl = ax.gridlines(draw_labels=True)
    gl.xlabels_top = gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    return fig, ax

In [ ]:
request = cimgt.OSM()  # Open Street Map

pad_lat = 0.002
pad_lon = 0.002

lon_min, lat_min, lon_max, lat_max = routes.bounds
extent = [lon_min - pad_lon, lon_max + pad_lon, lat_min - pad_lat, lat_max + pad_lat]

fig, ax = make_map(projection=request.crs)
ax.set_extent(extent)

img = ax.add_image(request, 14)

for route in routes.routes:
    s = ax.add_geometries(route.shape(), ccrs.PlateCarree(),
                        facecolor='none',
                        edgecolor='crimson',
                        linewidth=1)